In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, spacy

from string import punctuation
from pandarallel import pandarallel


2023-06-28 23:08:29.818258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 23:08:29.987237: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-28 23:08:30.694017: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bluesun/devel/lib:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu:/home/

In [2]:
train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')

train_df

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
idx = 0
fp = train_df['first_party'][idx]
sp = train_df['second_party'][idx]
facts = train_df['facts'][idx]
fact_tokens = nlp(facts)

print('first_party: ', fp)
print('second_party: ', sp)
print('facts: ', facts)


first_party:  Phil A. St. Amant
second_party:  Herman A. Thompson
facts:  On June 27, 1962, Phil St. Amant, a candidate for public office, made a television speech in Baton Rouge, Louisiana.  During this speech, St. Amant accused his political opponent of being a Communist and of being involved in criminal activities with the head of the local Teamsters Union.  Finally, St. Amant implicated Herman Thompson, an East Baton Rouge deputy sheriff, in a scheme to move money between the Teamsters Union and St. Amant’s political opponent. 
Thompson successfully sued St. Amant for defamation.  Louisiana’s First Circuit Court of Appeals reversed, holding that Thompson did not show St. Amant acted with “malice.”  Thompson then appealed to the Supreme Court of Louisiana.  That court held that, although public figures forfeit some of their First Amendment protection from defamation, St. Amant accused Thompson of a crime with utter disregard of whether the remarks were true.  Finally, that court hel

In [19]:
from prettytable import PrettyTable

def inspect_token(fact_tokens):
    table = PrettyTable(['TEXT', 'POS', 'TAG', 'DEP', 'HEAD', 'CHILDREN'])
    for token in fact_tokens:
        table.add_row([token.text, token.pos_, token.tag_, token.dep_, token.head.lemma_, str([c.text for c in token.children])])
    print(table)

In [21]:
inspect_token(fact_tokens)

+--------------+-------+------+----------+------------+----------------------------------------------------------+
|     TEXT     |  POS  | TAG  |   DEP    |    HEAD    |                         CHILDREN                         |
+--------------+-------+------+----------+------------+----------------------------------------------------------+
|      On      |  ADP  |  IN  |   prep   |    make    |                         ['June']                         |
|     June     | PROPN | NNP  |   pobj   |     on     |                   ['27', ',', '1962']                    |
|      27      |  NUM  |  CD  |  nummod  |    June    |                            []                            |
|      ,       | PUNCT |  ,   |  punct   |    June    |                            []                            |
|     1962     |  NUM  |  CD  |  nummod  |    June    |                            []                            |
|      ,       | PUNCT |  ,   |  punct   |    make    |                         

In [ ]:
from prettytable import PrettyTable

fp_name_list = [n for n in fp.split() if len(n)>1]
sp_name_list = [n for n in sp.split() if len(n)>1]

def inspect_token(fact_tokens):
    table = PrettyTable(['TEXT', 'POS', 'TAG', 'DEP', 'HEAD'])
    for token in fact_tokens:
        

        table.add_row([token.text, token.pos_, token.tag_, token.dep_, token.head.lemma_])
    print(table)

('file', 895),
 ('hold', 820),
 ('argue', 740),
 ('violate', 651),
 ('find', 627),
 ('sue', 526),
 ('deny', 522),
 ('allege', 468),
 ('have', 456),
 ('convict', 436),
 ('affirm', 434),
 ('appeal', 418),
 ('claim', 393),
 ('grant', 390),
 ('seek', 361),
 ('require', 356),
 ('rule', 342),
 ('dismiss', 331),
 ('reverse', 319),
 ('make', 293),

In [ ]:
legal_verbs = {'argue': 740,
               'violate': 651,
               'sue': 526,
               'allege': 468,
               'convict': 436,
               'claim': 393,
               'accuse': 391,
               'implicate': 387,}

